In [2]:
! unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip 
! unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                


In [3]:
! pip install transformers
# filter the data, take the same portion of each sense
# save the best weight ,means less loss valued trained batch 
# do some warm up step initially to make it more effiecient

In [4]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import transformers
from tqdm.notebook import tqdm

In [5]:
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased") 
bert_model.trainable = False

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

2022-01-22 07:17:18.973034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 07:17:18.974024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 07:17:18.974687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 07:17:18.975550: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
batch_size=32
max_len=128
EPOCHS=2

In [27]:
data=pd.read_csv("./train.csv",nrows=75000)
test_labels=pd.read_csv("./test_labels.csv",nrows=75000)
# train_data.head() 
# print(data.loc[0])
# data.columns

# data["toxic"].value_counts()
data.head()
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [28]:
row_no=list(data.shape)[0]
data["comment_text"]=data["comment_text"].map(lambda x:re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))

In [29]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 
sense_count_pd=pd.DataFrame(data[label_cols].value_counts()) 
sense_count_pd

0
toxic severe_toxic obscene threat insult identity_hate       
0     0            0       0      0      0              67289
1     0            0       0      0      0               2719
                   1       0      1      0               1799
                                  0      0                831
                   0       0      1      0                598
      1            1       0      1      0                473
      0            1       0      1      1                296
0     0            1       0      0      0                140
                   0       0      1      0                133
1     1            1       0      1      1                111
0     0            1       0      1      0                 88
1     1            1       0      0      0                 73
      0            1       1      1      0                 68
                   0       0      0      1                 58
                                  1      1                 55
                           1      0      0                 54
      1            1       1      1      0                 38
      0            1       1      1      1                 27
0     0            0       0      0      1                 25
1     1            0       0      0      0                 18
                   1       1      1      1                 17
      0            1       0      0      1                 15
0     0            0       0      1      1                 10
                           1      0      0                  9
                   1       0      1      1                  9
1     1            0       1      0      0                  8
      0            0       1      1      0                  7
                   1       1      0      0                  7
      1            1       0      0      1                  4
                   0       0      1      0                  4
                                         1                  3
      0            0       1      1      1                  3
      1            0       0      0      1                  2
0     0            0       1      1      0                  2
1     1            1       1      0      0                  2
      0            0       1      0      1                  2
0     0            1       0      0      1                  1
                           1      0      0                  1
                                  1      0                  1

In [30]:
labels =  test_labels[label_cols].values

In [31]:
# indexes = np.arange(12)
# np.array(labels[indexes], dtype="int32")
labels

array([[-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       ...,
       [-1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1],
       [ 0,  0,  0,  0,  0,  0]])

In [32]:
from sklearn.model_selection import train_test_split 
input_sen=data["comment_text"].values
# print(input_sen)
train_inputs,validation_inputs,train_labels,validation_labels=train_test_split(input_sen,labels,random_state=0,test_size=0.2) 


print(train_inputs.shape)
print(train_labels.shape)

print(validation_inputs.shape)
print(validation_labels.shape)
 

(60000,)
(60000, 6)
(15000,)
(15000, 6)


In [33]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence): 
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
         
        
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_tensors="tf",
        )   

        bert_output = bert_model(**encoded)
        
        sequence_output = bert_output.last_hidden_state 
         
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return sequence_output, labels
        else:
            return sequence_output

    def on_epoch_end(self): 
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [34]:
train_dataset=BertSemanticDataGenerator(train_inputs,train_labels,shuffle=True)
validation_dataset=BertSemanticDataGenerator(validation_inputs,validation_labels,shuffle=False)

In [ ]:
# for d in tqdm(train_dataset):
#     print(d)

In [15]:
input_layer = tf.keras.layers.Input(shape=(128, 768), name=None)  
flat=tf.keras.layers.Flatten()(input_layer) 
output = tf.keras.layers.Dense(6, activation="softmax")(flat)
model = tf.keras.models.Model(inputs=input_layer, outputs=output)
    
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 768)]        0         
_________________________________________________________________
flatten (Flatten)            (None, 98304)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 589830    
Total params: 589,830
Trainable params: 589,830
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)

In [ ]:
for step,(x_batch_train,labels) in enumerate(tqdm(train_dataset)):
    print(labels.flatten().shape)

In [ ]:
label

In [35]:
# custom training loop 
## update at each train step
## reset at the end of each batch

import time
## defining a optimizer 
optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3)

## defining loss function 
loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# ## mean loss define
# train_loss=tf.keras.metrics.Mean(name="train_loss")
# validation_loss=tf.keras.metrics.Mean(name="validation_loss")

# Metric
## dfining the accuracy metric to track our model accuracy.Here for 6 class we 
## have to declare 2d darray of row 6
train_acc_metric=[tf.keras.metrics.CategoricalAccuracy() for i in range(len(label_cols))]

val_acc_metric=[tf.keras.metrics.CategoricalAccuracy() for i in range(len(label_cols))]

# actually from logits denoting the probability from our custom model layed for each label.It is being fetched before the softmax layer to calculate loss between actual and predicted

batch_size=32
EPOCH=2
train_dataset_size=60000
validation_dataset_size=15000

@tf.function
def train_step(model,x_train,label):
    # Gradiane tape actually records the operation run in forward step
    with tf.GradientTape() as tape:
        #caluculate logits for comparison
        logits_prob=model(x_train,training=True)
        # calculate loss value 
        loss_value=loss_fn(label,logits_prob)
    #calculate gradient of trainable variables against the loss
    gradients=tape.gradient(loss_value,model.trainable_weights)
    # update the gradient according to gradient descent
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    # update the mean train ing loss
    train_loss(loss_value)
    # update accuracy metric for each of the 6 classes 
    for i,auc in enumerate(train_acc_metric):
        auc.update_state(label[:,i],logits_prob[:,i])
    return loss_value

@tf.function
def validation_step(model,x_validation,label):
    with tf.GradientTape() as tape:
        validation_logit_prob=model(x_validation,training=False)
        valid_loss=loss_fn(label,validation_logit_prob)
        validation_loss(valid_loss)
        for i,auc in enumerate(val_acc_metric):
            auc.update_state(label[:,i],validation_logit_prob[:,i]) 

def train_model(model,train_dataset,validation_dataset):
    for epoch in range(EPOCHS):
        print('\n Epoch No %d\n' % (epoch,))

        ### training part ###
        for step,(x_batch_train,labels) in enumerate(tqdm(train_dataset)):
            training_loss=train_step(model,x_batch_train,labels)
            
            #log result at every 200 batches
            if step%20==0:
                print(f'\nTrain Step: {epoch}, Loss: {train_loss.result()}')
#                 print("Trainng loss at %d batch of data: %.4f"%(step,float(training_loss)))
                # training accuracy metric at end
                for i, label_name in enumerate(label_cols):
                    print(f"{label_name} roc_auc {train_acc_metric[i].result()}")
                    # reset the accuracy metric after every epoch
                    train_acc_metric[i].reset_states()
            
#         training_accuracy=train_acc_metric.result()
#         print("\nTraining accuracy after %d epoch : %.4f"%(epoch,training_accuracy))
#         train_acc_metric.reset_states()
        
        
        ### validation part ###
        for step,(x_batch_val,labels) in enumerate(tqdm(validation_dataset)):
            validation_step(model,x_batch_val,labels)
        print(f'\nTrain Step: {epoch}, Loss: {validation_loss.result()}')
        
        for i, label_name in enumerate(label_cols):
            print(f"{label_name} roc_auc {val_acc_metric[i].result()}") 
            val_acc_metric[i].reset_states()
#         validation_acc=val_acc_metric.result()
#         val_acc_metric.reset_states()
#         print("\n validation accuracy : %4.f"%(validation_acc))

train_model(model,train_dataset,validation_dataset)
model.save("my_custom_train_model.h5")


 Epoch No 0



  0%|          | 0/1875 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentrop


Train Step: 0, Loss: -6.22709321975708
toxic roc_auc 0.0
severe_toxic roc_auc 0.0
obscene roc_auc 0.0
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 0, Loss: -6.277719974517822
toxic roc_auc 0.05000000074505806
severe_toxic roc_auc 0.05000000074505806
obscene roc_auc 0.0
threat roc_auc 0.0
insult roc_auc 0.05000000074505806
identity_hate roc_auc 0.0

Train Step: 0, Loss: -6.255007266998291
toxic roc_auc 0.05000000074505806
severe_toxic roc_auc 0.0
obscene roc_auc 0.0
threat roc_auc 0.10000000149011612
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 0, Loss: -6.274140357971191
toxic roc_auc 0.0
severe_toxic roc_auc 0.0
obscene roc_auc 0.0
threat roc_auc 0.10000000149011612
insult roc_auc 0.0
identity_hate roc_auc 0.20000000298023224

Train Step: 0, Loss: -6.31077766418457
toxic roc_auc 0.05000000074505806
severe_toxic roc_auc 0.0
obscene roc_auc 0.05000000074505806
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 0, Lo

  0%|          | 0/468 [00:00<?, ?it/s]

...


Train Step: 0, Loss: -6.311530113220215
toxic roc_auc 0.04273504391312599
severe_toxic roc_auc 0.03846153989434242
obscene roc_auc 0.025641025975346565
threat roc_auc 0.03205128386616707
insult roc_auc 0.03418803587555885
identity_hate roc_auc 0.03846153989434242

 Epoch No 1



  0%|          | 0/1875 [00:00<?, ?it/s]


Train Step: 1, Loss: -6.3551836013793945
toxic roc_auc 0.06666667014360428
severe_toxic roc_auc 0.13333334028720856
obscene roc_auc 0.0
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 1, Loss: -6.3550825119018555
toxic roc_auc 0.25
severe_toxic roc_auc 0.05000000074505806
obscene roc_auc 0.0
threat roc_auc 0.10000000149011612
insult roc_auc 0.10000000149011612
identity_hate roc_auc 0.0

Train Step: 1, Loss: -6.351948261260986
toxic roc_auc 0.05000000074505806
severe_toxic roc_auc 0.0
obscene roc_auc 0.0
threat roc_auc 0.05000000074505806
insult roc_auc 0.0
identity_hate roc_auc 0.0

Train Step: 1, Loss: -6.351147651672363
toxic roc_auc 0.0
severe_toxic roc_auc 0.15000000596046448
obscene roc_auc 0.0
threat roc_auc 0.0
insult roc_auc 0.0
identity_hate roc_auc 0.20000000298023224

Train Step: 1, Loss: -6.352055549621582
toxic roc_auc 0.20000000298023224
severe_toxic roc_auc 0.05000000074505806
obscene roc_auc 0.05000000074505806
threat roc_auc 0.0
insult roc

  0%|          | 0/468 [00:00<?, ?it/s]


Train Step: 1, Loss: -6.292143821716309
toxic roc_auc 0.03846153989434242
severe_toxic roc_auc 0.03846153989434242
obscene roc_auc 0.025641025975346565
threat roc_auc 0.03418803587555885
insult roc_auc 0.03846153989434242
identity_hate roc_auc 0.044871795922517776


In [ ]:
# from keras.models import load_model
# model=load_model('../input/mymodel/my_model.h5')

In [38]:
s4 = 'fuck you bitch!!!'
sentence_pairs = np.array([s4])
test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

pro=model.predict(test_data)
print(pro)
print(label_cols[np.argmax(pro)])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[0.24514431 0.18649596 0.16171435 0.14327097 0.12693104 0.13644336]]
toxic


In [ ]:
# loading the pre-defined bert model weights
bert_model.Trainable=True

train_dataset=BertSemanticDataGenerator(train_inputs,train_labels,shuffle=True)
validation_dataset=BertSemanticDataGenerator(validation_inputs,validation_labels,shuffle=False)

trained_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)

In [ ]:
model.trainable_variables

In [ ]:
# submission_df=pd.read_csv("./sample_submission.csv",index_col='id')
# test_df=pd.read_csv("./test.csv")
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# print(submission_df.head())
# print(test_df.head())


# test_bert_op=BertSemanticDataGenerator(test_df['comment_text'],None,include_targets=False,shuffle=True)

In [ ]:
# for i,sen in enumerate(tqdm(test_bert_op)):
#     sample_ids = test_df.iloc[i*32:(i+1)*32]['id'] 
#     pred=model.predict(sen)
#     submission_df.loc[sample_ids, label_cols] = pred
#     print(pred)
#     print(sen.shape)

In [ ]:
# submission_df.to_csv('submission.csv')

In [ ]:
# sample_ids = test_df.iloc[0*32:(0+1)*32]['id'] 
# print(sample_ids)
# submission_df.loc[sample_ids]

In [ ]:
model.save("my_model.h5")

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# # to convert the iput ids array in same size(column no=max(column no))
# tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
# max_length=128
# bert_model=transformers.TFBertModel.from_pretrained("bert-base-uncased")
# bert_model.trainable=False

# def tokenize(data,tokenizer=tokenizer,max_length=max_length):
# #     input_ids=[]
# #     attention_masks=[]
#     bert_outputs=[]
#     for sentence in tqdm(data):
        
#         encoded_data=tokenizer.batch_encode_plus(
#                         sentence,
#                         add_special_tokens=True,
#                         max_length=max_length,
#                         truncation=True,
#                         return_attention_mask=True,
#                         return_token_type_ids=True,
#                         pad_to_max_length=True,
#                         return_tensors="tf",
#                     )

# #         input_id=np.array(encoded_data["input_ids"],dtype="int32")
# #         attention_mask=np.array(encoded_data["attention_mask"],dtype="int32") 
        
#         bert_output=bert_model(**encoded_data)
#         sequence_output = bert_output.last_hidden_state
#         bert_outputs.append(sequence_output)
#     return bert_outputs
# #         input_ids.append(input_id)
# #         attention_masks.append(attention_mask)
        
# #     return [input_ids,attention_masks]

# bert_op=tokenize(data['comment_text'])
# # input_ids=pad_sequences(bert_op[0],maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# # attention_masks=bert_op[1]
# # bert_op.shape

In [ ]:
# len(bert_op)

In [ ]:
# attention_masks=np.array(attention_masks)
# attention_masks=pad_sequences(attention_masks,maxlen=max_length,dtype='long',value=0,truncating="post",padding="post")
# input_ids.shape
# attention_masks.shape

In [ ]:
# # creating batched dataset
# epochs=2
# def create_batch_dataset(data,epochs=epochs,batch_size=batch_size,buffer_size=1000,train=True):
#     dataset=tf.data.Dataset.from_tensor_slices(data)
# #     print(dataset.as_numpy_iterator())
#     if train:
#         dataset=dataset.shuffle(buffer_size=buffer_size)
#         # uses for shuffling the dataset.Select the first buffer_size element from dataset
#     dataset=dataset.repeat(epochs)
#     # just repeat the whole dataset
#     dataset=dataset.batch(batch_size=batch_size)
#     # devide the whole dataset into batch size and create an array of array
#     if train:
#         dataset=dataset.prefetch(1)
#     #     It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two 
#     # elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 
#     # batches, of 20 examples each).
#     return dataset
# train_dataset=create_batch_dataset((train_inputs,train_masks,train_labels),train=True)
# validation_dataset=create_batch_dataset((validation_inputs,validation_masks,validation_labels),train=True)

In [ ]:
# print(train_dataset.as_numpy_iterator())
# print(validation_dataset.as_numpy_iterator())

In [ ]:
# list(train_dataset.as_numpy_iterator())[0][0].shape

In [ ]:
# for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):
#             print(token_ids.shape)

In [ ]:
# https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers
# https://www.kaggle.com/satyamkryadav/bert-model-96-77/notebook